### Properties

Мы увидели, как можно просто назначать и считывать значения атрибутов непосредственно для объекта:

In [1]:
import math

In [2]:
class Circle:
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return math.pi * self.radius ** 2

In [3]:
c = Circle(1)

In [4]:
c.radius

1

In [5]:
c.area()

3.141592653589793

И, конечно же, мы можем изменить радиус этого объекта напрямую:

In [6]:
c.radius = 2

In [7]:
c.radius

2

In [8]:
c.area()

12.566370614359172

Нежелательным моментом здесь является то, что мы можем задать отрицательное значение при создании экземпляра `Circle`.

In [9]:
c = Circle(-1)

In [10]:
c.radius, c.area()

(-1, 3.141592653589793)

Отрицательный радиус не имеет особого смысла — на самом деле, мы можем даже задать радиус в виде строки:

In [11]:
c = Circle('100')

In [12]:
c.radius

'100'

И наш метод `area()` завершится ошибкой при вызове:

In [13]:
try:
    c.area()
except TypeError as ex:
    print(ex)

unsupported operand type(s) for ** or pow(): 'str' and 'int'


И наш метод `area()` завершится ошибкой при вызове: Мы можем исправить это, выполнив некоторые проверки в методе `__init__`:

In [14]:
class Circle:
    def __init__(self, radius):
        if not (isinstance(radius, float) or isinstance(radius, int)):
            raise ValueError('radius must be an integer or a float')
        if radius < 0:
            raise ValueError('radius cannot be negative')
        self.radius = radius

    def area(self):
        return math.pi * self.radius ** 2

In [15]:
c = Circle(1)

In [16]:
c = Circle(1.5)

In [17]:
try:
    c = Circle('100')
except ValueError as ex:
    print(ex)

radius must be an integer or a float


In [18]:
try:
    c = Circle(-10)
except ValueError as ex:
    print(ex)

radius cannot be negative


Итак, это решает эту проблему. Но теперь у нас также есть проблема, что мы можем изменить радиус, **после** создания экземпляра, на что-то неприемлемое:

In [19]:
c = Circle(1)

In [20]:
c.radius = 'hello'

Поскольку атрибут `radius` является всего лишь прямым атрибутом экземпляра, мы мало что можем с этим поделать. (Кроме того, если у вас есть опыт работы с другими языками программирования, в которых есть нечто, называемое закрытой областью действия, в Python ничего подобного нет — все открыто).

Python предоставляет альтернативный механизм этим «голым» атрибутам — так называемые **свойства**.

Свойства определяются с помощью функций для **установки** значения свойства и **получения** значения свойства.

Кроме того, мы можем определить свойства, которые имеют только геттер (то есть свойство только для чтения) или, что встречается гораздо реже, только геттер (то есть свойство только для записи), которые мы здесь рассматривать не будем.

Начнем со свойств, доступных только для чтения.

In [21]:
class Circle:
    def __init__(self, radius):
        if not (isinstance(radius, float) or isinstance(radius, int)):
            raise ValueError('radius must be an integer or a float')
        if radius < 0:
            raise ValueError('radius cannot be negative')
        self._radius = radius

    @property
    def radius(self):
        print('radius getter called...')
        return self._radius

Обратите внимание, что мы по-прежнему храним значение радиуса в «голом» атрибуте класса, но мы используем **соглашение**, предваряющее имя атрибута подчеркиванием. Это означает (по **соглашению**), что атрибут является «приватным» для реализации класса. Любой, кто использует наш класс, поймет, что ему не следует изменять этот атрибут напрямую (но он все равно может, если захочет. Однако мы не виноваты, если он что-то испортит! :-) ).

Обратите внимание, что когда **декоратор** `property` применяется к методу экземпляра, **имя метода** будет определять **имя свойства**.

In [22]:
c = Circle(10)

In [23]:
c.radius

radius getter called...


10

Кстати, мы также можем использовать свойство только для чтения для `area`, что позволяет нам использовать `c.area` вместо `c.area()` — не очень удобно, но синтаксис более приятный:

In [24]:
class Circle:
    def __init__(self, radius):
        if not (isinstance(radius, float) or isinstance(radius, int)):
            raise ValueError('radius must be an integer or a float')
        if radius < 0:
            raise ValueError('radius cannot be negative')
        self._radius = radius

    @property
    def radius(self):
        print('radius getter called...')
        return self._radius

    @property
    def area(self):
        print('property area called...')
        return math.pi * self.radius ** 2

Обратите внимание, как я использовал `self.radius` в свойстве `area` - это фактически вызовет геттер свойства для `radius`. Вместо этого мы могли бы использовать `self._radius`, но поскольку у нас есть геттер свойства, мы обычно используем само свойство, а не "вспомогательную" переменную.

In [25]:
c = Circle(1)

In [26]:
c.area

property area called...
radius getter called...


3.141592653589793

Итак, теперь у нас по сути есть «неизменяемый» класс — мы можем задать радиус при создании экземпляра, но нет ничего, что позволило бы нам изменить свойство радиуса **после** создания экземпляра. (хотя технически это неверно, поскольку мы всегда можем изменить `_radius` напрямую — но мы не должны этого делать из-за этого начального подчеркивания).

А что, если мы хотим разрешить пользователям изменять радиус?

Для этого мы можем написать свойство **setter**. Для этого мы снова используем декоратор и метод экземпляра.

In [27]:
class Circle:
    def __init__(self, radius):
        if not (isinstance(radius, float) or isinstance(radius, int)):
            raise ValueError('radius must be an integer or a float')
        if radius < 0:
            raise ValueError('radius cannot be negative')
        self._radius = radius

    @property
    def radius(self):
        print('radius getter called...')
        return self._radius

    @radius.setter
    def radius(self, value):
        print('radius setter called...')
        if not (isinstance(value, float) or isinstance(value, int)):
            raise ValueError('radius must be an integer or a float')
        if value < 0:
            raise ValueError('radius cannot be negative')
        self._radius = value

    @property
    def area(self):
        print('property area called...')
        return math.pi * self.radius ** 2

Обратите внимание на синтаксис декоратора — он **должен** использовать имя свойства, которое мы использовали для **сеттера** свойства, и имя функции также **должно** совпадать.

In [28]:
c = Circle(1)

In [29]:
c.radius = 2

radius setter called...


In [30]:
c.radius

radius getter called...


2

И теперь мы защищены от установки неверного значения `radius` после создания экземпляра:

In [31]:
try:
    c.radius = "100"
except ValueError as ex:
    print(ex)

radius setter called...
radius must be an integer or a float


Вы заметите, что у нас один и тот же код проверки как в `__init__`, так и в установщике радиуса.

Мы можем избавиться от проверок в `__init__`, если просто используем установщик свойств в `__init__`:

In [32]:
class Circle:
    def __init__(self, radius):
        self.radius = radius

    @property
    def radius(self):
        print('radius getter called...')
        return self._radius

    @radius.setter
    def radius(self, value):
        print('radius setter called...')
        if not (isinstance(value, float) or isinstance(value, int)):
            raise ValueError('radius must be an integer or a float')
        if value < 0:
            raise ValueError('radius cannot be negative')
        self._radius = value

    @property
    def area(self):
        print('property area called...')
        return math.pi * self.radius ** 2

Посмотрите, как мы использовали сеттер свойств в методе `__init__`, используя `self.radius = radius` — это фактически вызывает сеттер:

In [33]:
c = Circle(10)

radius setter called...


И это значит, что наш init по-прежнему защищен:

In [34]:
try:
    c = Circle(-100)
except ValueError as ex:
    print(ex)

radius setter called...
radius cannot be negative


One word of caution:

Ниже приведена ошибка, часто допускаемая при начале использования свойств:

In [35]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        return self.radius

Обратите внимание, как `self.radius` используется **внутри** функции-получателя `radius`?

Какой метод вызывается, когда мы читаем `c.radius`?

...метод `radius()`.

Итак, когда мы пишем `self.radius` внутри метода `radius()`, мы на самом деле вызываем метод `radius()` снова и снова и снова — это называется бесконечной рекурсией.

Давайте посмотрим, что произойдет:

In [36]:
c = Circle(1)

In [37]:
c.radius

RecursionError: maximum recursion depth exceeded

Итак, если вы когда-либо увидите это исключение при использовании геттера (или сеттера), то вы, скорее всего, вызываете геттер изнутри геттера или сеттер изнутри сеттера, что будет выглядеть примерно так:

In [38]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, value):
        self.radius = value  # this actually calls the setter!

In [39]:
c = Circle(1)

In [40]:
c.radius

1

In [41]:
c.radius = 2

RecursionError: maximum recursion depth exceeded